In [259]:
using BenchmarkTools
using Mmap

In [109]:
function todisk(fname::String,arr::Array)
    temp = reshape(arr,1,:)
    open(fname,"w") do io
        write(io,temp)
    end
end

todisk (generic function with 1 method)

In [133]:
function fromdisk(fname,d1,d2,ex)
    dt = eltype(ex)
    load = open(read,fname)
    out = reinterpret(dt,load)
    return convert(Array{dt},out)
end

fromdisk (generic function with 2 methods)

In [136]:
function roundtrip(fname::String,array)
    d1 = size(array)[1]
    d2 = size(array)[2]
    todisk(fname,array)
    return fromdisk(fname,d1,d2,array[1])
end

roundtrip (generic function with 1 method)

In [252]:
a = randn(Float32,10000000)
b = randn(Float32,10000000)

todisk("A",a)

todisk("B",b)

40000000

In [253]:
function mydot(a1::String,a2::String,bufsize::Int,ex)
    "Implementation of disk based dot product using fixed buffers"
    dt = typeof(ex)
    bytes = sizeof(dt)
    print(bytes)
    sum = 0.0
    buf1 = zeros(dt,bufsize)
    buf2 = zeros(dt,bufsize)
    io1 = open(a1,"r")
    io2 = open(a2,"r")
    seekend(io1)
    seekend(io2)
    len1 = position(io1)
    len2 = position(io2)
    seekstart(io1)
    seekstart(io2)
    steps = floor(Int,len1/(bufsize*bytes))
    for i in collect(UnitRange(1,steps))
        buf1[:] = convert(Array{dt},reinterpret(dt,read(io1,bytes*bufsize)))
        buf2[:] = convert(Array{dt},reinterpret(dt,read(io2,bytes*bufsize)))
        for j in collect(UnitRange(1,bufsize))
            sum += buf1[j]*buf2[j]
        end
    end
    close(io1)
    close(io2)
    return sum
end

mydot (generic function with 2 methods)

In [254]:
mydot("A","B",10000,convert(Float32,1.0))

4

-5086.647221928334

In [255]:
dot(a,b)

-5086.6953f0

In [278]:
@benchmark mydot("A","B",10000)

BenchmarkTools.Trial: 
  memory estimate:  191.17 MiB
  allocs estimate:  6019
  --------------
  minimum time:     30.369 ms (11.44% GC)
  median time:      30.940 ms (12.95% GC)
  mean time:        31.253 ms (12.96% GC)
  maximum time:     40.665 ms (16.20% GC)
  --------------
  samples:          160
  evals/sample:     1

In [279]:
@benchmark dot(a,b)

BenchmarkTools.Trial: 
  memory estimate:  16 bytes
  allocs estimate:  1
  --------------
  minimum time:     2.858 ms (0.00% GC)
  median time:      2.952 ms (0.00% GC)
  mean time:        3.018 ms (0.00% GC)
  maximum time:     6.025 ms (0.00% GC)
  --------------
  samples:          1654
  evals/sample:     1

In [256]:
A = randn(100,102)

100×102 Array{Float64,2}:
 -1.13103    -0.63955    0.576073   0.877694   …   0.452681     -1.73208   
 -0.912902    0.519288  -0.262308   0.0929026      1.11514      -0.751076  
 -0.255074    0.074724  -1.30985   -0.776638       1.31479       0.231601  
 -1.2808     -1.35968   -0.725662  -0.699938      -0.464256      0.711422  
  0.113623   -0.4986     0.349596  -0.515931      -0.0699194     0.386732  
 -1.62416     0.604924   1.74733   -1.35256    …  -1.08527       0.327488  
  0.276763    0.974647   0.887323  -0.181051       1.82202      -1.97294   
 -0.355001   -0.657703  -0.185762  -0.159571      -0.941186     -0.587083  
  0.602469    2.17951   -0.601548   1.01991        0.567288     -0.187882  
 -1.45535     0.214453  -0.482803  -3.16234        0.53201      -0.014058  
  0.215323    0.821513  -0.20291   -0.262372   …  -2.65003       1.46019   
 -0.0265786  -1.08371    1.83358    2.11669        1.47607       0.0837261 
 -0.61286    -0.452371   0.319428  -0.336284       0.178866   

In [257]:

s = open("/tmp/mmap.bin","w+")
write(s,size(A,1))
write(s,size(A,2))
write(s,A)
close(s)

In [263]:
s = open("/tmp/mmap.bin")
m = read(s,Int)
n = read(s,Int)
A2 = Mmap.mmap(s,Matrix{Float64},(m,n))

close(s)

In [272]:
j = open("/tmp/b.bin","w+")
write(j,size(b,1))
write(j,b)
close(j)

In [265]:
k = open("/tmp/a.bin","w+")
write(k,size(a,1))
write(k,a)
close(k)

In [270]:
close(j)

In [275]:
j = open("/tmp/b.bin","r")
m = read(j, Int)
aa = Mmap.mmap(j,Array{Float32,1},(m))

10000000-element Array{Float32,1}:
 -0.054233607
 -0.9499914  
  0.6584976  
  0.20736992 
 -0.025722919
 -0.3679929  
  0.15126811 
  1.4166478  
 -0.7332895  
 -0.48933688 
  0.46667036 
 -0.6293265  
  0.9716014  
  ⋮          
  1.9068085  
  0.52789885 
 -0.21519543 
  3.3688467  
  1.2195475  
  1.6440344  
  1.2348168  
  1.5600194  
  1.1300931  
 -0.54702646 
 -2.0609894  
 -2.8397446  

In [276]:
k = open("/tmp/a.bin","r")
n = read(k,Int)
bb = Mmap.mmap(k,Array{Float32,1},n)

10000000-element Array{Float32,1}:
 -0.74406934
  0.8486048 
  0.42535952
 -0.9300928 
 -1.5610851 
  0.5714895 
  0.53098154
  0.06357608
 -0.9778362 
  1.7052861 
  0.4992941 
 -2.2918475 
 -1.389691  
  ⋮         
  0.13229272
  0.11427516
  0.18035567
  0.47115222
 -0.3229812 
 -2.9791162 
 -0.8643453 
  0.67145944
 -1.9363497 
 -0.99873394
 -1.4255662 
  0.80592686

In [277]:

@benchmark dot(aa,bb)

BenchmarkTools.Trial: 
  memory estimate:  16 bytes
  allocs estimate:  1
  --------------
  minimum time:     2.879 ms (0.00% GC)
  median time:      2.955 ms (0.00% GC)
  mean time:        3.030 ms (0.00% GC)
  maximum time:     5.800 ms (0.00% GC)
  --------------
  samples:          1647
  evals/sample:     1

In [280]:
dot(aa,bb)

-5086.6953f0